## Apartamentos

In [2]:
pip install unidecode

     |████████████████████████████████| 235 kB 7.1 MB/s 


In [3]:
import pandas as pd 
from unidecode import unidecode

apartamentos = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/dados_consolidados/imoveis_apart.csv")
apartamentos = apartamentos[["Classe", "UF", "Município", "Endereço", 'format_address', "Bairro", "Regime de Utilização", "Área do Terreno Total", "Área da União", "lat", "lng"]]
apartamentos = apartamentos.rename(columns = {"Classe": "classe", "UF": "uf", "Município": "municipio", "Endereço": "endereco", "Bairro": "bairro", "Regime de Utilização": "regime_utilizacao",
                               "Área do Terreno Total": "area_terreno_total", "Área da União": "area_uniao", 'format_address': "endereco_formatado"})
apartamentos = apartamentos[apartamentos["municipio"]!="EXTERIOR"]
apartamentos["area_terreno_total"] = apartamentos["area_terreno_total"].str.replace(".", "").str.replace(",", ".").astype(float)
apartamentos["area_uniao"] = apartamentos["area_uniao"].str.replace(".", "").str.replace(",", ".").astype(float)
apartamentos["mun_for_pred"] = apartamentos["municipio"].str.lower().str.replace(" ", "")
apartamentos["mun_for_pred"] = apartamentos["mun_for_pred"].apply(unidecode)

In [4]:
import pandas as pd
from unidecode import unidecode

data_apartamentos = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/html_apartamentos/data_apartamentos.csv")
data_apartamentos = data_apartamentos[["municipio", "area", "lat", "lng", "valor"]]

data_apartamentos = data_apartamentos.drop_duplicates()
data_apartamentos["municipio"] = data_apartamentos["municipio"].str.lower()
data_apartamentos["area"] = data_apartamentos["area"].str.replace("\n", "")
data_apartamentos["valor"] = data_apartamentos["valor"].str.replace("\n", "")
data_apartamentos = data_apartamentos[~data_apartamentos["area"].str.contains("-")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("A partir de ")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Mês")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Sob Consulta")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Dia")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Semana")]
data_apartamentos["valor"] = data_apartamentos["valor"].str.replace(".", "").str.replace("R", "").str.replace('[\$,]', '', regex=True).astype(float)
data_apartamentos["area"] = data_apartamentos["area"].astype(float)
data_apartamentos["mun_for_pred"] = data_apartamentos["municipio"].str.lower().str.replace(" ", "")
data_apartamentos["mun_for_pred"] = data_apartamentos["mun_for_pred"].apply(unidecode)

In [5]:
import pickle
import math
import numpy as np
from tqdm import tqdm

apartamentos["preco_predict"] = "-"
muns = ["brasilia", "belohorizonte", "saopaulo", "riodejaneiro", "saoluis", "portoalegre", "salvador", "manaus", "fortaleza", "goiania"]

for mun in muns:

    with open(f"/content/drive/MyDrive/alethea_stn/models/apartamentos/mdl_apart_{mun}.pkl", 'rb') as f:
        mdl = pickle.load(f)

    train = data_apartamentos[data_apartamentos["mun_for_pred"]==mun]
    train['valor'] = train['valor'].map(lambda x: math.log(x))

    X = train.drop(["valor", "municipio", "mun_for_pred"], 1)
    y = train["valor"]

    mdl.fit(X, y)

    x_pred = apartamentos[apartamentos["mun_for_pred"]==mun][["area_uniao", "lat", "lng"]].rename(columns = {"area_uniao": "area"})
    y_pred = mdl.predict(x_pred)
    y_pred = np.exp(y_pred)

    x_pred["y_pred"] = y_pred

    for r in tqdm(x_pred.index, desc = f"Input predict dos preços da cidade {mun}"):
      apartamentos["preco_predict"].loc[r] = round(x_pred["y_pred"].loc[r], 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
Input predict dos preços da cidade brasilia:   0%|          | 0/8834 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
Input predict dos preços da cidade goiania: 100%|██████████| 3/3 [00:00<00:00, 898.59it/s]


In [6]:
apartamentos

,classe,uf,municipio,endereco,endereco_formatado,bairro,regime_utilizacao,area_terreno_total,area_uniao,lat,lng,mun_for_pred,preco_predict
0,IMOVEL FUNCIONAL,PE,RECIFE,R JOSE MARIA DE MIRANDA 177 APARTAMENTO 101 - ...,"R. José Maria de Miranda, 177 - 101 - Boa Viag...",BOA VIAGEM,Imóvel Funcional,1644.89,1644.89,-8.108503,-34.893753,recife,-
1,IMOVEL FUNCIONAL,PE,RECIFE,R JOSE MARIA DE MIRANDA 177 APARTAMENTO 102 - ...,"R. José Maria de Miranda, 177 - 102 - Boa Viag...",BOA VIAGEM,Imóvel Funcional,1644.89,1644.89,-8.108503,-34.893753,recife,-
2,IMOVEL FUNCIONAL,PE,RECIFE,R JOSE MARIA DE MIRANDA 177 APARTAMENTO 201 - ...,"R. José Maria de Miranda, 177 - 201 - Boa Viag...",BOA VIAGEM,Imóvel Funcional,1644.89,1644.89,-8.108503,-34.893753,recife,-
3,IMOVEL FUNCIONAL,PE,RECIFE,R JOSE MARIA DE MIRANDA 177 APARTAMENTO 202 - ...,"R. José Maria de Miranda, 177 - 202 - Boa Viag...",BOA VIAGEM,Imóvel Funcional,1644.89,1644.89,-8.108503,-34.893753,recife,-
4,IMOVEL FUNCIONAL,PE,RECIFE,R JOSE MARIA DE MIRANDA 177 APARTAMENTO 301 - ...,"R. José Maria de Miranda, 177 - 301 - Boa Viag...",BOA VIAGEM,Imóvel Funcional,1644.89,1644.89,-8.108503,-34.893753,recife,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9482,USO ESPECIAL,DF,BRASILIA,"ST SRI 2 HFA 104 BLOCO "" F "" DO SETOR DE EXPAN...","Brasília - Cruzeiro / Sudoeste / Octogonal, Br...",SETOR SUDOESTE,Vago para uso,139.00,139.00,-15.801854,-47.939240,brasilia,1.67311e+06
9483,USO ESPECIAL,DF,BRASILIA,"ST SRI 2 HFA 105 BLOCO "" F "" DO SETOR DE EXPAN...","Brasília - Cruzeiro / Sudoeste / Octogonal, Br...",SETOR SUDOESTE,Vago para uso,139.00,139.00,-15.801854,-47.939240,brasilia,1.67311e+06
9484,USO ESPECIAL,DF,BRASILIA,"ST SRI 2 HFA 106 BLOCO "" F "" DO SETOR DE EXPAN...","Brasília - Cruzeiro / Sudoeste / Octogonal, Br...",SETOR SUDOESTE,Vago para uso,139.00,139.00,-15.801854,-47.939240,brasilia,1.67311e+06
9485,USO ESPECIAL,DF,BRASILIA,"ST SRI 2 HFA 107 BLOCO "" F "" DO SETOR DE EXPAN...","Brasília - Cruzeiro / Sudoeste / Octogonal, Br...",SETOR SUDOESTE,Vago para uso,139.00,139.00,-15.801854,-47.939240,brasilia,1.67311e+06


## Residências 

In [7]:
import pandas as pd 
from unidecode import unidecode

residen = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/dados_consolidados/imoveis_residen.csv")
residen = residen[["Classe", "UF", "Município", "Endereço", 'format_address', "Bairro", "Regime de Utilização", "Área do Terreno Total", "Área da União", "lat", "lng"]]
residen = residen.rename(columns = {"Classe": "classe", "UF": "uf", "Município": "municipio", "Endereço": "endereco", "Bairro": "bairro", "Regime de Utilização": "regime_utilizacao",
                               "Área do Terreno Total": "area_terreno_total", "Área da União": "area_uniao", 'format_address': "endereco_formatado"})
residen = residen[residen["municipio"]!="EXTERIOR"]
residen = residen[~residen["area_terreno_total"].str.contains('#')]
residen["area_terreno_total"] = residen["area_terreno_total"].str.replace(".", "").str.replace(",", ".").astype(float)
residen["area_uniao"] = residen["area_uniao"].str.replace(".", "").str.replace(",", ".").astype(float)
residen["mun_for_pred"] = residen["municipio"].str.lower().str.replace(" ", "")
residen["mun_for_pred"] = residen["mun_for_pred"].apply(unidecode)

In [8]:
data_residen = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/html_residen/data_residen.csv")
data_residen = data_residen[["municipio", "area", "lat", "lng", "valor"]]

data_residen = data_residen.drop_duplicates()
data_residen["municipio"] = data_residen["municipio"].str.lower()
data_residen["area"] = data_residen["area"].str.replace("\n", "")
data_residen["valor"] = data_residen["valor"].str.replace("\n", "")
data_residen = data_residen[~data_residen["area"].str.contains("-")]
data_residen = data_residen[~data_residen["valor"].str.contains("A partir de ")]
data_residen = data_residen[~data_residen["valor"].str.contains("Mês")]
data_residen = data_residen[~data_residen["valor"].str.contains("Sob Consulta")]
data_residen= data_residen[~data_residen["valor"].str.contains("Dia")]
data_residen = data_residen[~data_residen["valor"].str.contains("Semana")]
data_residen["valor"] = data_residen["valor"].str.replace(".", "").str.replace("R", "").str.replace('[\$,]', '', regex=True).astype(float)
data_residen["area"] = data_residen["area"].astype(float)
data_residen["mun_for_pred"] = data_residen["municipio"].str.lower().str.replace(" ", "")
data_residen["mun_for_pred"] = data_residen["mun_for_pred"].apply(unidecode)

In [9]:
import pickle
import math
import numpy as np
from tqdm import tqdm

residen["preco_predict"] = "-"
muns = ["brasilia", "belem", "belohorizonte", "saopaulo", "riodejaneiro", "saoluis", "maceio", "portoalegre",
        "curitiba", "salvador", "manaus", "fortaleza", "goiania"]

for mun in muns:

    with open(f"/content/drive/MyDrive/alethea_stn/models/residencias/mdl_residen_{mun}.pkl", 'rb') as f:
        mdl = pickle.load(f)

    train = data_residen[data_residen["mun_for_pred"]==mun]
    train['valor'] = train['valor'].map(lambda x: math.log(x))

    X = train.drop(["valor", "municipio", "mun_for_pred"], 1)
    y = train["valor"]

    mdl.fit(X, y)

    x_pred = residen[residen["mun_for_pred"]==mun][["area_uniao", "lat", "lng"]].rename(columns = {"area_uniao": "area"})
    y_pred = mdl.predict(x_pred)
    y_pred = np.exp(y_pred)

    x_pred["y_pred"] = y_pred

    for r in tqdm(x_pred.index, desc = f"Input predict dos preços da cidade {mun}"):
      residen["preco_predict"].loc[r] = round(x_pred["y_pred"].loc[r], 2)
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
Input predict dos preços da cidade brasilia:   0%|          | 0/746 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
Input predict dos preços da cidade goiania: 100%|██████████| 57/57 [00:00<00:00, 4383.49it/s]


In [18]:
residen

,classe,uf,municipio,endereco,endereco_formatado,bairro,regime_utilizacao,area_terreno_total,area_uniao,lat,lng,mun_for_pred,preco_predict
0,IMOVEL FUNCIONAL,RO,GUAJARA-MIRIM,AV COSTA MARQUES 465 ALOJAMENTO - DA UA/G.MIRI...,"Costa Marques, RO, 76937-000, Brazil",CENTRO,Imóvel Funcional,1000.00,1000.00,-12.435720,-64.229452,guajara-mirim,-
1,IMOVEL FUNCIONAL,PI,SIMPLICIO MENDES,R PROJETO SERTANEJO DE SIMPLICIO MENDES 00480 ...,"R. Jose do Moura Fe, Simplício Mendes - PI, 64...",CENTRO,Imóvel Funcional,840.00,840.00,-7.861082,-41.906225,simpliciomendes,-
2,IMOVEL FUNCIONAL,MA,ZE DOCA,R OSVALDO CRUZ 85 CEP: 65365000,"Zé Doca - State of Maranhão, 65365-000, Brazil",CENTRO,Imóvel Funcional,295.87,295.87,-3.232985,-46.132678,zedoca,-
3,IMOVEL FUNCIONAL,MA,ZE DOCA,R OSVALDO CRUZ 95 CEP: 65365000,"R. Osvaldo Cruz, 95 - Centro, Santa Cruz do Su...",CENTRO,Imóvel Funcional,295.87,295.87,-29.708816,-52.426026,zedoca,-
4,IMOVEL FUNCIONAL,MA,ZE DOCA,R OSVALDO CRUZ 108 CEP: 65365000,"R. Osvaldo Cruz, 108, Bacabal - MA, 65700-000,...",CENTRO,Imóvel Funcional,295.87,295.87,-4.228582,-44.777376,zedoca,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6311,USO ESPECIAL,MT,MARCELANDIA,R LONDRINA 480 CEP: 78535000,"R. Londrina, 480, Marcelândia - MT, 78535-000,...",CENTRO,"Cessão para Prefeituras, Estados e outras Enti...",6232.50,6232.50,-11.082211,-54.517957,marcelandia,-
6312,USO ESPECIAL,MT,MARCELANDIA,R GETÚLIO BISPO DOS SANTOS 1162 CASA CEP: 7853...,"R. Getúlio Bpo. dos Santos, 1162, Marcelândia ...",CENTRO,"Cessão para Prefeituras, Estados e outras Enti...",525.00,525.00,-11.084108,-54.527159,marcelandia,-
6313,USO ESPECIAL,MT,TERRA NOVA DO NORTE,AV MATO GROSSO S/N CEP: 78505000,"Av. Mato Grosso - Terra Nova do Norte, MT, 785...",CENTRO,"Cessão para Prefeituras, Estados e outras Enti...",19517.00,19517.00,-10.602443,-55.118845,terranovadonorte,-
6314,USO ESPECIAL,MT,LUCAS DO RIO VERDE,R CAMPO ERE S/N CASA CEP: 78455000,"R. Campo Erê - Pioneiro, Lucas do Rio Verde - ...",PIONEIRO,"Cessão para Prefeituras, Estados e outras Enti...",1372.00,1372.00,-13.055639,-55.909966,lucasdorioverde,-


## Terrenos

In [10]:
import pandas as pd 
from unidecode import unidecode

terrenos = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/dados_consolidados/imoveis_terrenos.csv")
terrenos = terrenos[["Classe", "UF", "Município", "Endereço", 'format_address', "Bairro", "Regime de Utilização", "Área do Terreno Total", "Área da União", "lat", "lng"]]
terrenos = terrenos.rename(columns = {"Classe": "classe", "UF": "uf", "Município": "municipio", "Endereço": "endereco", "Bairro": "bairro", "Regime de Utilização": "regime_utilizacao",
                               "Área do Terreno Total": "area_terreno_total", "Área da União": "area_uniao", 'format_address': "endereco_formatado"})
terrenos = terrenos[terrenos["municipio"]!="EXTERIOR"]
terrenos = terrenos[~terrenos["area_terreno_total"].str.contains('#')]
terrenos["area_terreno_total"] = terrenos["area_terreno_total"].str.replace(".", "").str.replace(",", ".").astype(float)
terrenos["area_uniao"] = terrenos["area_uniao"].str.replace(".", "").str.replace(",", ".").astype(float)
terrenos["mun_for_pred"] = terrenos["municipio"].str.lower().str.replace(" ", "")
terrenos["mun_for_pred"] = terrenos["mun_for_pred"].apply(unidecode)

In [11]:
data_terrenos = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/html_terrenos/data_terrenos.csv")
data_terrenos = data_terrenos[["municipio", "area", "lat", "lng", "valor"]]
data_terrenos = data_terrenos.drop_duplicates()
data_terrenos["municipio"] = data_terrenos["municipio"].str.lower()
data_terrenos["area"] = data_terrenos["area"].str.replace("\n", "")
data_terrenos["valor"] = data_terrenos["valor"].str.replace("\n", "")
data_terrenos = data_terrenos[~data_terrenos["area"].str.contains("-")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("A partir de ")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Mês")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Sob Consulta")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Dia")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Semana")]
data_terrenos["valor"] = data_terrenos["valor"].str.replace(".", "").str.replace("R", "").str.replace('[\$,]', '', regex=True).astype(float)
data_terrenos["area"] = data_terrenos["area"].astype(float)
data_terrenos["mun_for_pred"] = data_terrenos["municipio"].str.lower().str.replace(" ", "")
data_terrenos["mun_for_pred"] = data_terrenos["mun_for_pred"].apply(unidecode)

In [12]:
import pickle
import math
import numpy as np
from tqdm import tqdm

terrenos["preco_predict"] = "-"
muns = ["brasilia", "belem", "belohorizonte", "saopaulo", "riodejaneiro", "saoluis", "maceio", "portoalegre",
        "curitiba", "salvador", "manaus", "fortaleza", "goiania"]

for mun in muns:

    with open(f"/content/drive/MyDrive/alethea_stn/models/residencias/mdl_residen_{mun}.pkl", 'rb') as f:
        mdl = pickle.load(f)

    train = data_terrenos[data_terrenos["mun_for_pred"]==mun]
    train['valor'] = train['valor'].map(lambda x: math.log(x))

    X = train.drop(["valor", "municipio", "mun_for_pred"], 1)
    y = train["valor"]

    mdl.fit(X, y)

    x_pred = terrenos[terrenos["mun_for_pred"]==mun][["area_uniao", "lat", "lng"]].rename(columns = {"area_uniao": "area"})
    y_pred = mdl.predict(x_pred)
    y_pred = np.exp(y_pred)

    x_pred["y_pred"] = y_pred

    for r in tqdm(x_pred.index, desc = f"Input predict dos preços da cidade {mun}"):
      terrenos["preco_predict"].loc[r] = round(x_pred["y_pred"].loc[r], 2)
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
Input predict dos preços da cidade brasilia:   0%|          | 0/1016 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
Input predict dos preços da cidade goiania: 100%|██████████| 11/11 [00:00<00:00, 2109.43it/s]


In [21]:
terrenos

,classe,uf,municipio,endereco,endereco_formatado,bairro,regime_utilizacao,area_terreno_total,area_uniao,lat,lng,mun_for_pred,preco_predict
0,DOMINIAL,AM,MANAUS,R ZEBU 1 ANTIGA EST DA FRIGOMASA CEP: 69073670,"Estr. de Acesso Frigomasa - Vila Buriti, Manau...",COL OLIVEIRA MACHADO,OCUPACAO,3601.60,3601.60,-3.152885,-59.992864,manaus,1.13301e+06
1,DOMINIAL,AM,MANAUS,R ZEBU 1 ANTIGA ESTRADA DA FRIGOMASA CEP: 6907...,"Estr. de Acesso Frigomasa - Vila Buriti, Manau...",COL OLIVEIRA MACHADO,OCUPACAO,20640.00,20640.00,-3.152885,-59.992864,manaus,1.13301e+06
2,DOMINIAL,AM,MANAUS,R ZEBU 1 ANTIGA ESTRADA DA FRIGOMASA CEP: 6907...,"Estr. de Acesso Frigomasa - Vila Buriti, Manau...",COLONIA OLIVEIRA MAC,OCUPACAO,17038.56,17038.56,-3.152885,-59.992864,manaus,1.13301e+06
3,DOMINIAL,AM,MANAUS,R RAUL FOLLEEREOU 108 CEP: 69008320,R. Dr. Raoul Follereau - Colônia Antônio Aleix...,COLONIA ANTONIO ALEI,OCUPACAO,17759.58,3827.84,-3.089022,-59.885463,manaus,1.19283e+06
4,DOMINIAL,AM,MANAUS,R PADRE MARIO S/N CEP: 69008120,"R. Padre Mario - Colônia Antônio Aleixo, Manau...",COLONIA ANTONIO ALEI,OCUPACAO,22713.13,1739.38,-3.088380,-59.885918,manaus,1.19283e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37266,USO ESPECIAL,DF,BRASILIA,LT SCES TRECHO 3 8 PROJETO ORLA - PÓLO 08 CEP:...,SCES Trecho 3 - St. de Clubes Esportivos Sul -...,ASA SUL,Vago para uso,27581.00,27581.00,-15.811093,-47.852547,brasilia,5.23708e+06
37267,USO ESPECIAL,DF,BRASILIA,Q SAFS QUADRA SETOR DE ADMINISTRACAO FEDER 01 ...,"St. de Administração Federal Sul - Brasília, D...",ZONA CIVICO ADMINIST,Vago para uso,6369.91,6369.91,-15.807534,-47.867116,brasilia,5.27677e+06
37268,USO ESPECIAL,DF,BRASILIA,Q SAFS QUADRA SETOR DE ADMINISTRACAO FEDER 01 ...,"St. de Administração Federal Sul - Brasília, D...",ZONA CIVICO ADMINIST,Vago para uso,6304.60,6304.60,-15.807534,-47.867116,brasilia,5.27677e+06
37269,USO ESPECIAL,DF,BRASILIA,Q SAFS QUADRA SETOR DE ADMINISTRACAO FEDER 03 ...,SAFS Quadra 2 - St. de Administração Federal S...,ZONA CIVICO ADMINIST,Vago para uso,6000.00,6000.00,-15.804544,-47.869367,brasilia,5.11381e+06


## Consolidação dos dados

In [13]:
apartamentos["tipo_imovel"] = "Apartamento"
residen["tipo_imovel"] = "Residência/Casa"
terrenos["tipo_imovel"] = "Terreno"

data_imoveis_precos = pd.concat([apartamentos, residen, terrenos])
data_imoveis_precos = data_imoveis_precos.reset_index(drop=True)[["classe", "tipo_imovel", "uf", "municipio", "endereco", "bairro", "regime_utilizacao", "area_terreno_total", "area_uniao", "lat", "lng", "preco_predict"]]

In [14]:
data_imoveis_precos.to_csv("/content/drive/MyDrive/alethea_stn/data/dados_consolidados/data_imoveis_precos.csv")

In [177]:
data_apartamentos = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/html_apartamentos/data_apartamentos.csv")

data_apartamentos = data_apartamentos.drop_duplicates()
data_apartamentos = data_apartamentos.drop(["metrica"], 1)
data_apartamentos["area"] = data_apartamentos["area"].str.replace("\n", "")
data_apartamentos["valor"] = data_apartamentos["valor"].str.replace("\n", "")
data_apartamentos["uf"] = data_apartamentos["uf"].str.replace("\n", "")
data_apartamentos = data_apartamentos[~data_apartamentos["area"].str.contains("-")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("A partir de ")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Mês")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Sob Consulta")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Dia")]
data_apartamentos = data_apartamentos[~data_apartamentos["valor"].str.contains("Semana")]
data_apartamentos["valor"] = data_apartamentos["valor"].str.replace(".", "").str.replace("R", "").str.replace('[\$,]', '', regex=True).astype(float)
data_apartamentos["area"] = data_apartamentos["area"].astype(float)


data_apartamentos["municipio"] = data_apartamentos["municipio"].replace({"RiodeJaneiro": "Rio de Janeiro", "BeloHorizonte": "Belo Horizonte",
                                        "PortoAlegre": "Porto Alegre", "SãoLuís": "São Luís", "SãoPaulo": "São Paulo"})

data_apartamentos["tipo_imovel"] = "Apartamento"
data_apartamentos = data_apartamentos[["tipo_imovel", "uf", "municipio", "endereco", "format_address", "lat", "lng", "area", "valor"]]
data_apartamentos

,tipo_imovel,uf,municipio,endereco,format_address,lat,lng,area,valor
0,Apartamento,RJ,Rio de Janeiro,"Rua Senador Nabuco - Vila Isabel, Rio de Janei...","R. Sen. Nabuco - Vila Isabel, Rio de Janeiro -...",-22.913830,-43.252564,94.0,390000.0
2,Apartamento,RJ,Rio de Janeiro,"Avenida Jaime Poggi, 99 - Jacarepaguá, Rio de ...","Av. Jaime Poggi, 99 - Jacarepaguá, Rio de Jane...",-22.968900,-43.391418,83.0,559000.0
3,Apartamento,RJ,Rio de Janeiro,"Rua Cinco de Julho - Copacabana, Rio de Janeir...","R. 5 de Julho - Copacabana, Rio de Janeiro - R...",-22.971108,-43.190286,82.0,900000.0
4,Apartamento,RJ,Rio de Janeiro,"Rua Povina Cavalcanti - São Conrado, Rio de Ja...","Rua Povina Cavalcanti - São Conrado, Rio de Ja...",-22.996820,-43.259414,200.0,1700000.0
5,Apartamento,RJ,Rio de Janeiro,"Avenida Nossa Senhora de Copacabana, 861 - Cop...","Av. Nossa Sra. de Copacabana, 861 - Copacabana...",-22.974553,-43.188991,46.0,529000.0
...,...,...,...,...,...,...,...,...,...
40563,Apartamento,RJ,Rio de Janeiro,"Rua Santos Titara, 123 - Méier, Rio de Janeiro...","R. Santos Titara, 123 - Todos os Santos, Rio d...",-22.900410,-43.286029,60.0,425000.0
40566,Apartamento,RJ,Rio de Janeiro,"Rua General Ribeiro da Costa, 32 - Leme, Rio d...","Rua General Ribeiro da Costa, 32 - Leme, Rio d...",-22.961740,-43.168661,51.0,700000.0
40567,Apartamento,RJ,Rio de Janeiro,Avenida Prefeito Dulcídio Cardoso - Barra da T...,"Av. Pref. Dulcídio Cardoso - Barra da Tijuca, ...",-23.005037,-43.351836,131.0,1500000.0
40568,Apartamento,RJ,Rio de Janeiro,"Rua Prudente de Morais - Ipanema, Rio de Janei...","R. Prudente de Morais - Ipanema, Rio de Janeir...",-22.985266,-43.206307,178.0,2500000.0


In [178]:
data_residen = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/html_residen/data_residen.csv")

data_residen = data_residen.drop_duplicates()
data_residen["area"] = data_residen["area"].str.replace("\n", "")
data_residen["valor"] = data_residen["valor"].str.replace("\n", "")
data_residen["uf"] = data_residen["uf"].str.replace("\n", "")
data_residen = data_residen[~data_residen["area"].str.contains("-")]
data_residen = data_residen[~data_residen["valor"].str.contains("A partir de ")]
data_residen = data_residen[~data_residen["valor"].str.contains("Mês")]
data_residen = data_residen[~data_residen["valor"].str.contains("Sob Consulta")]
data_residen= data_residen[~data_residen["valor"].str.contains("Dia")]
data_residen = data_residen[~data_residen["valor"].str.contains("Semana")]
data_residen["valor"] = data_residen["valor"].str.replace(".", "").str.replace("R", "").str.replace('[\$,]', '', regex=True).astype(float)
data_residen["area"] = data_residen["area"].astype(float)

data_residen["municipio"] = data_residen["municipio"].replace({"RiodeJaneiro": "Rio de Janeiro", "BeloHorizonte": "Belo Horizonte",
                                        "PortoAlegre": "Porto Alegre", "SãoLuís": "São Luís", "SãoPaulo": "São Paulo"})

data_residen["tipo_imovel"] = "Residência/Casa"
data_residen = data_residen[["tipo_imovel", "uf", "municipio", "endereco", "format_address", "lat", "lng", "area", "valor"]]
data_residen


,tipo_imovel,uf,municipio,endereco,format_address,lat,lng,area,valor
0,Residência/Casa,SP,São Paulo,"Rua Rio Verde - Vila Palmeiras, São Paulo - SP","R. Rio Verde, São Paulo - SP, 02675-031, Brazil",-23.488563,-46.706424,250.0,780000.0
1,Residência/Casa,SP,São Paulo,"Estrada José Ribeiro Leite, 413 - IPATINGA, So...","Estr. José Ribeiro Leite, 413, Sorocaba - SP, ...",-23.497860,-47.538604,47.0,202900.0
2,Residência/Casa,SP,São Paulo,"Rua Professor Antônio Filgueiras de Lima, 234 ...","Rua Professor Antônio Filgueiras de Lima, 234 ...",-23.567470,-46.748754,90.0,250000.0
3,Residência/Casa,SP,São Paulo,"Rua Benjamim do Monte, 138 - Parque Cruzeiro d...","Rua Benjamim do Monte, 138 - Parque Cruzeiro d...",-23.495798,-46.460352,40.0,390000.0
4,Residência/Casa,SP,São Paulo,"Rua Plínio Mota, 59 - Vila Prudente, São Paulo...","R. Plínio Mota, 59 - Vila Tolstoi, São Paulo -...",-23.598068,-46.527932,240.0,1500000.0
...,...,...,...,...,...,...,...,...,...
30046,Residência/Casa,RJ,Rio de Janeiro,"Rua das Azaléas, 395 - Vila Valqueire, Rio de...","R. das Azáleas, 395 - Vila Valqueire, Rio de J...",-22.882526,-43.365987,495.0,1800000.0
30047,Residência/Casa,RJ,Rio de Janeiro,"Rua Costa Ferraz, 21 - Rio Comprido, Rio de Ja...","R. Costa Ferraz, 21 - Rio Comprido, Rio de Jan...",-22.923351,-43.204484,220.0,600000.0
30048,Residência/Casa,RJ,Rio de Janeiro,"Rua Soldado Arlindo da Silva, 180 - Guaratiba,...","R. Sd. Arlíndo da Silva, 180 - Guaratiba, Rio ...",-23.004736,-43.629175,80.0,270000.0
30049,Residência/Casa,RJ,Rio de Janeiro,"Rua São Platão, 70 - Anil, Rio de Janeiro - RJ","R. São Platão, 70 - Anil, Rio de Janeiro - RJ,...",-22.947355,-43.336799,172.0,980000.0


In [179]:
data_terrenos = pd.read_csv("/content/drive/MyDrive/alethea_stn/data/html_terrenos/data_terrenos.csv")

data_terrenos = data_terrenos.drop_duplicates()
data_terrenos["area"] = data_terrenos["area"].str.replace("\n", "")
data_terrenos["valor"] = data_terrenos["valor"].str.replace("\n", "")
data_terrenos["uf"] = data_terrenos["uf"].str.replace("\n", "")
data_terrenos = data_terrenos[~data_terrenos["area"].str.contains("-")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("A partir de ")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Mês")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Sob Consulta")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Dia")]
data_terrenos = data_terrenos[~data_terrenos["valor"].str.contains("Semana")]
data_terrenos["valor"] = data_terrenos["valor"].str.replace(".", "").str.replace("R", "").str.replace('[\$,]', '', regex=True).astype(float)
data_terrenos["area"] = data_terrenos["area"].astype(float)

data_terrenos["municipio"] = data_terrenos["municipio"].replace({"RiodeJaneiro": "Rio de Janeiro", "BeloHorizonte": "Belo Horizonte",
                                        "PortoAlegre": "Porto Alegre", "SãoLuís": "São Luís", "SãoPaulo": "São Paulo"})

data_terrenos["tipo_imovel"] = "Terreno"
data_terrenos = data_terrenos[["tipo_imovel", "uf", "municipio", "endereco", "format_address", "lat", "lng", "area", "valor"]]
data_terrenos

,tipo_imovel,uf,municipio,endereco,format_address,lat,lng,area,valor
0,Terreno,SP,São Paulo,"Rua Bruno Bertucci, 257 - Freguesia do Ó, São ...","Rua Bruno Bertucci, 257 - Vila Arcadia, São Pa...",-23.503525,-46.699398,154.0,670000.0
2,Terreno,SP,São Paulo,"Avenida José Maria da Silva Paranhos, 99 - Jar...","Av. José Maria da Silva Paranhos, 99 - Vila So...",-23.604821,-46.735001,274.0,1350000.0
3,Terreno,SP,São Paulo,"Rua Filipe da Silva, 130 - Lauzane Paulista, S...","R. Filipe da Silva, 130 - Mandaqui, São Paulo ...",-23.482449,-46.642285,324.0,1300000.0
4,Terreno,SP,São Paulo,"Rua Capitary - Jardim São Carlos, São Paulo - SP","Rua Capitary - Jardim Sao Carlos (Zona Leste),...",-23.516483,-46.474820,93.0,370000.0
5,Terreno,SP,São Paulo,"Rua Diogo Rodrigues, 63 - Parque Ipê, São Paul...","R. Diogo Rodrigues, 63 - Parque Ipe, São Paulo...",-23.641767,-46.765637,175.0,500000.0
...,...,...,...,...,...,...,...,...,...
30155,Terreno,MG,Belo Horizonte,"Rua Virgílio Machado - Barreiro, Belo Horizont...","R. Virgílio Machado - Teixeira Dias, Belo Hori...",-19.984155,-44.021487,190.0,600000.0
30156,Terreno,MG,Belo Horizonte,"Nova Suíssa, Belo Horizonte - MG","Nova Suíça, Belo Horizonte - State of Minas Ge...",-19.933067,-43.977952,300.0,890000.0
30157,Terreno,MG,Belo Horizonte,"Ouro Preto, Belo Horizonte - MG","Ouro Preto, Belo Horizonte - State of Minas Ge...",-19.870936,-43.983305,271.0,1070000.0
30158,Terreno,MG,Belo Horizonte,"Copacabana, Belo Horizonte - MG","Parque Copacabana, Belo Horizonte - State of M...",-19.833198,-43.987855,31.0,144000.0


In [181]:
data_imoveis_scrapping = pd.concat([data_apartamentos, data_residen, data_terrenos])

In [184]:
data_imoveis_scrapping.reset_index(drop=True).to_csv("/content/drive/MyDrive/alethea_stn/data/dados_consolidados/data_imoveis_scrapping.csv")